In [2]:
import cv2
from cv2 import data
import mediapipe as mp
import imutils
import pandas as pd

In [5]:
#video frame name
video_frame_name = "data"

In [ ]:
name = []
for i in range(33):
    name.append(str(i) + "_x")
    name.append(str(i) + "_y")
    name.append(str(i) + "_z")
    name.append(str(i) + "_vis")
df = pd.DataFrame(columns=name)

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture("data/" + video_frame_name + ".mp4")

with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("saving data...")
            df.to_csv(r'C:/Users/sufiyan/Documents/exercise tracker/data/' +
                      video_frame_name+".csv", index=False, header=True)
            print("data captured successfully")
            break

        image = imutils.resize(image, width=400)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = pose.process(image)
        coordinate = results.pose_landmarks.landmark

        keypoints = []
        x = []
        y = []
        z = []
        vis = []

        for data_point in results.pose_landmarks.landmark:
            keypoints.append({
                'X': data_point.x,
                'Y': data_point.y,
                'Z': data_point.z,
                'Visibility': data_point.visibility,
            })
            x.append(-data_point.x)
            y.append(-data_point.y)
            z.append(-data_point.z)
            vis.append(data_point.visibility)

        data = {}
        for i in range(33):
            data[str(i) + "_x"] = x[i]
            data[str(i) + "_y"] = y[i]
            data[str(i) + "_z"] = z[i]
            data[str(i) + "_vis"] = vis[i]

        df = df.append(data, ignore_index=True)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow('MediaPipe Pose', image)
        key = cv2.waitKey(1) & 0xff
        if key == 27: 
            break


cap.release()
